In [ ]:
#TODO Feel free to change the method definition and documentation, you can create the method that you need

#Method for computing the new centroids. The parameters are:
#     - centroids. Current centroids.
#     - data. data[0] will contain the image
#             data[1] will contain the current label or centroid
# Returns the new centroids. The data and labels do not change.
def compute_centroids(centroids,data):
    #For each centroid, calculate the new center, by finding the mean of the data points in each channel.

    return None

In [ ]:
#TODO Feel free to change the method definition and documentation, you can create the method that you need

#Method for assigning the points to a new centroid. The parameters are:
#     - centroids. Current centroids.
#     - data. data[0] will contain the image
#             data[1] will contain the current label or centroid
# Returns the new labels. The points are assigned based on Euclidean distance.
def assign(centroids,data):

    return None

In [ ]:
#TODO Feel free to change the method definition and documentation, you can create the method that you need

#Method that implements K-Means. The parameters are:
#     - k. The problem with K-Means is that it needs a hyperparameter k, selected by the user.
#     - image. 
#Returns a segmented image.
def k_means(k,image):
    #Generate the centroids. Randomly. Our centroids are three dimensional.
    centroids = {i+1:[np.random.randint(255),np.random.randint(255),np.random.randint(255)]for i in range(k)}
    
    return new_i

In [ ]:
def compute_centroids(centroids, data):
    new_centroids = np.zeros_like(centroids)

    for i in range(centroids.shape[0]):
        # Find all the pixels that belong to the i-th centroid
        idx = (data[1] == i)
        pixels = data[0][idx]

        # Check if the cluster is empty
        if pixels.size == 0:
            # If the cluster is empty, set its centroid to a random RGB value
            new_centroids[i] = [np.random.randint(0, 255) for _ in range(3)]
        else:
            # Compute the mean of the pixel values in each channel
            new_center = np.mean(pixels, axis=0)

            # Store the new centroid
            new_centroids[i] = new_center

    return new_centroids

def assign(centroids, data):
    # Initialize an array to store the new labels
    new_labels = np.zeros_like(data[1])

    # Loop over each pixel in the image
    for i in range(data[0].shape[0]):
        for j in range(data[0].shape[1]):
            # Get the RGB values of the pixel
            pixel = data[0][i,j]

            # Compute the distances to each centroid using the Euclidean distance metric
            distances = np.linalg.norm(pixel - centroids, axis=1)

            # Assign the pixel to the centroid with the minimum distance
            new_labels[i,j] = np.argmin(distances)

    return new_labels
def k_means(k, image):
    # Initialize the centroids randomly
    centroids = np.random.randint(0, 255, size=(k, 3))

    # Initialize the labels array with zeros
    labels = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

    # Initialize the old_labels array with ones to enter the loop
    old_labels = np.ones_like(labels)

    # Loop until the labels converge
    while not np.array_equal(old_labels, labels):
        # Assign each pixel to the nearest centroid
        data = (image, labels)
        new_labels = assign(centroids, data)

        # Update the centroids based on the new labels
        data = (image, new_labels)
        new_centroids = compute_centroids(centroids, data)

        # Update the old labels and centroids
        old_labels = labels
        centroids = new_centroids
        labels = new_labels

    # Map each label to a unique RGB color
    colors = np.array([centroids[i] for i in range(k)])
    new_i = colors[labels]

    return new_i

In [2]:
pip install torch


Note: you may need to restart the kernel to use updated packages.


In [7]:
from skimage import io
from skimage import color

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
%matplotlib inline
import requests
import torch 

In [3]:
url = "https://www.rainforest-alliance.org/wp-content/uploads/2021/06/poison-dart-frog-thumb-1-400x400.jpg"
img_data = requests.get(url).content
with open('image.jpg', 'wb') as handler:
    handler.write(img_data)
img = io.imread("image.jpg")

In [15]:
import torch
import numpy as np

def compute_centroids(centroids, data):
    new_centroids = torch.zeros_like(centroids)

    for i in range(centroids.shape[0]):
        # Find all the pixels that belong to the i-th centroid
        idx = (data[1] == i)
        pixels = data[0][idx]

        # Check if the cluster is empty
        if pixels.numel() == 0:
            # If the cluster is empty, set its centroid to a random RGB value
            new_centroids[i] = torch.randint(0, 255, size=(1, 3)).float()
        else:
            # Compute the mean of the pixel values in each channel
            new_center = pixels.float().mean(dim=0)

            # Store the new centroid
            new_centroids[i] = new_center

    return new_centroids

def assign(centroids, data):
    # Convert the input image from a NumPy array to a PyTorch tensor
    image = torch.from_numpy(data[0])

    # Get the RGB values of all the pixels in the image
    pixels = image.permute(2, 0, 1).float()

    # Compute the distances to each centroid using the Euclidean distance metric
    distances = torch.cdist(pixels.view(3, -1).T, centroids.float())

    # Assign each pixel to the centroid with the minimum distance
    new_labels = torch.argmin(distances, dim=1)

    # Reshape the labels back into the original image shape
    new_labels = new_labels.view(data[0].shape[:2])

    return new_labels

def k_means(k, image):
    # Convert the input image from a NumPy array to a PyTorch tensor
    img_tensor = torch.from_numpy(image)

    # Initialize the centroids randomly
    centroids = torch.randint(0, 255, size=(k, 3)).float()

    # Initialize the labels array with zeros
    labels = torch.zeros(image.shape[0], image.shape[1], dtype=torch.uint8)
    # Initialize the old_labels array with ones to enter the loop
    old_labels = torch.ones_like(labels, dtype=torch.uint8)


    # Loop until the labels converge
    while not torch.equal(old_labels, labels):
        # Assign each pixel to the nearest centroid
        data = (image, labels)
        new_labels = assign(centroids, data)

        # Update the centroids based on the new labels
        data = (img_tensor, new_labels)
        new_centroids = compute_centroids(centroids, data)

        # Update the old labels and centroids
        old_labels = labels
        centroids = new_centroids
        labels = new_labels

    # Map each label to a unique RGB color
    colors = centroids.round().clamp(0, 255).to(torch.uint8)
    new_i = colors[labels]

    return new_i.permute(1, 2, 0).numpy()


In [16]:
images = []
for k in range(1,9):
    images.append(k_means(k,img))

RuntimeError: Expected object of scalar type unsigned char but got scalar type __int64 for argument 'other'